In [130]:
import tensorflow as tf
import numpy as np
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
def zero_pad(X, pad):
    """
    Pad with zeros all images of the dataset X. The padding is applied to the height and width of an image, 
    as illustrated in Figure 1.
    
    Argument:
    X -- python numpy array of shape (m, n_H, n_W, n_C) representing a batch of m images
    pad -- integer, amount of padding around each image on vertical and horizontal dimensions
    
    Returns:
    X_pad -- padded image of shape (m, n_H + 2*pad, n_W + 2*pad, n_C)
    """
    
    ### START CODE HERE ### (≈ 1 line)
    
    if type(pad) == int:
        
        if pad == 0:
            return X
        if pad > 0:
            return np.pad(X,((0,0),(pad,pad),(pad,pad),(0,0)),mode='symmetric')#'constant',constant_values = 0)
    else:
        
        if len(pad) == 2:
            pad_h, pad_w = pad
        return np.pad(X,((0,0),pad_h,pad_w,(0,0)),mode='symmetric')#'constant',constant_values = 0)   
    

In [138]:
# GRADED FUNCTION: pool_forward

def pool_forward(A_prev, hparameters, mode = "max"):
    """
    Implements the forward pass of the pooling layer
    
    Arguments:
    A_prev -- Input data, numpy array of shape (m, n_H_prev, n_W_prev, n_C_prev)
    hparameters -- python dictionary containing "f" and "stride"
    mode -- the pooling mode you would like to use, defined as a string ("max" or "average")
    
    Returns:
    A -- output of the pool layer, a numpy array of shape (m, n_H, n_W, n_C)
    cache -- cache used in the backward pass of the pooling layer, contains the input and hparameters 
    """
    nceil = lambda x: int(np.ceil(x))
                      
    # Retrieve dimensions from the input shape
    (m, n_H_prev, n_W_prev, n_C_prev) = A_prev.shape
    
    # Retrieve hyperparameters from "hparameters"
    f = hparameters["f"]
    stride = hparameters["stride"]
    pad = hparameters['pad']
    # Define the dimensions of the output
    
    n_H = int(1 + (n_H_prev - f) / stride)
    n_W = int(1 + (n_W_prev - f) / stride)
    n_C = n_C_prev
    
    if pad == 'valid':
        pad = 0
    elif pad == 'same':
        n_H = nceil(n_H_prev/ stride)
        n_W = nceil(n_W_prev / stride)
        pad_h = (n_H - 1)*stride - n_H_prev + f
        pad_w = (n_W - 1)*stride - n_W_prev + f
        
        if pad_h/2==0:
            p_h_t = p_h_d = nceil(pad_h/2)
        else:
            if f % 2 == 1:
                p_h_t, p_h_d = nceil(pad_h/2),int(pad_h/2)
            if f % 2 == 0:
                p_h_t, p_h_d = int(pad_h/2),nceil(pad_h/2)
                
        if pad_w/2==0:
            p_w_l = p_w_r = int(pad_w/2)
        else:
            if f % 2 == 1:
                p_w_l , p_w_r = nceil(pad_w/2),int(pad_w/2)
            if f % 2 == 0:
                p_w_l , p_w_r = int(pad_w/2),nceil(1+pad_w/2)
        
        
        print(0,pad,A_prev.shape,(n_H,n_W),(pad_h,pad_w))
        pad = ((p_h_t,p_h_d),(p_w_l,p_w_r))
    
    # Initialize output matrix A
    
    A_prev_pad = zero_pad(A_prev,pad)
    A = np.zeros((m, n_H, n_W, n_C))
    
    
    ### START CODE HERE ###
    for i in range(m):                         # loop over the training examples
        for h in range(n_H):                     # loop on the vertical axis of the output volume
            for w in range(n_W):                 # loop on the horizontal axis of the output volume
                for c in range (n_C):            # loop over the channels of the output volume
                    
                    # Find the corners of the current "slice" (≈4 lines)
                    vert_start = (h)*stride
                    vert_end = vert_start+f
                    horiz_start = (w)*stride
                    horiz_end = horiz_start+f
                    
                    # Use the corners to define the current slice on the ith training example of A_prev, channel c. (≈1 line)
                    a_prev_slice = A_prev_pad[i,vert_start:vert_end,horiz_start:horiz_end,c]
                    
                    #print(a_prev_slice.shape)
                    # Compute the pooling operation on the slice. Use an if statment to differentiate the modes. Use np.max/np.mean.
                    if mode == "max":
                        A[i, h, w, c] = np.max(a_prev_slice)
                    elif mode == "average":
                        A[i, h, w, c] = np.mean(a_prev_slice)
    
    ### END CODE HERE ###
    
    # Store the input and hparameters in "cache" for pool_backward()
    cache = (A_prev, hparameters)
    
    # Making sure your output shape is correct
    assert(A.shape == (m, n_H, n_W, n_C))
    
    return A, cache

In [139]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
tfx = tf.placeholder(tf.float32,shape=(None,None,None,None))
hparameters = {'stride':1,'f':3}
same1 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='SAME')
same2 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='VALID')

with tf.Session() as sess:
    a1 = sess.run(same1, feed_dict={tfx:x})
    a2 = sess.run(same2, feed_dict={tfx:x})
    print('origin\n',x[2,:,:,1])
    print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
    print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[1.1294839  1.1294839  0.41005164 0.37756377]
 [1.1294839  1.2245077  1.2245077  1.2245077 ]
 [1.1294839  1.2245077  1.2245077  1.2245077 ]
 [0.698032   1.2245077  1.2245077  1.2245077 ]
 [0.16938244 0.16938244 0.40890053 0.40890053]] 
 shape (3, 5, 4, 2)
valid
 [[1.2245077 1.2245077]
 [1.2245077 1.2245077]
 [1.2245077 1.2245077]] 
 shape (3, 3, 2, 2)


In [140]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
a1,_ = pool_forward(x,{'stride':1,'f':3,'pad':'same'})
a2,_ = pool_forward(x,{'stride':1,'f':3,'pad':'valid'})

print('origin\n',x[2,:,:,1])
print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

0 same (3, 5, 4, 2) (5, 4) (2, 2)
origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[1.12948391 1.12948391 0.41005165 0.37756379]
 [1.12948391 1.2245077  1.2245077  1.2245077 ]
 [1.12948391 1.2245077  1.2245077  1.2245077 ]
 [0.69803203 1.2245077  1.2245077  1.2245077 ]
 [0.16938243 0.16938243 0.40890054 0.40890054]] 
 shape (3, 5, 4, 2)
valid
 [[1.2245077 1.2245077]
 [1.2245077 1.2245077]
 [1.2245077 1.2245077]] 
 shape (3, 3, 2, 2)


In [141]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
tfx = tf.placeholder(tf.float32,shape=(None,None,None,None))
hparameters = {'stride':1,'f':2}
same1 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='SAME')
same2 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='VALID')

with tf.Session() as sess:
    a1 = sess.run(same1, feed_dict={tfx:x})
    a2 = sess.run(same2, feed_dict={tfx:x})
    print('origin\n',x[2,:,:,1])
    print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
    print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[ 1.1294839   0.41005164  0.37756377  0.37756377]
 [ 1.1294839   1.2245077   1.2245077   0.5935785 ]
 [ 0.698032    1.2245077   1.2245077   0.5935785 ]
 [ 0.16938244  0.03261455  0.40890053  0.40890053]
 [-0.85951596 -1.3122834   0.40890053  0.40890053]] 
 shape (3, 5, 4, 2)
valid
 [[1.1294839  0.41005164 0.37756377]
 [1.1294839  1.2245077  1.2245077 ]
 [0.698032   1.2245077  1.2245077 ]
 [0.16938244 0.03261455 0.40890053]] 
 shape (3, 4, 3, 2)


In [142]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
a1,_ = pool_forward(x,{'stride':1,'f':2,'pad':'same'})
a2,_ = pool_forward(x,{'stride':1,'f':2,'pad':'valid'})

print('origin\n',x[2,:,:,1])
print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

0 same (3, 5, 4, 2) (5, 4) (1, 1)
origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[ 1.12948391  0.41005165  0.37756379  0.37756379]
 [ 1.12948391  1.2245077   1.2245077   0.59357852]
 [ 0.69803203  1.2245077   1.2245077   0.59357852]
 [ 0.16938243  0.03261455  0.40890054  0.40890054]
 [-0.85951594 -1.31228341  0.40890054  0.40890054]] 
 shape (3, 5, 4, 2)
valid
 [[1.12948391 0.41005165 0.37756379]
 [1.12948391 1.2245077  1.2245077 ]
 [0.69803203 1.2245077  1.2245077 ]
 [0.16938243 0.03261455 0.40890054]] 
 shape (3, 4, 3, 2)


In [88]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
tfx = tf.placeholder(tf.float32,shape=(None,None,None,None))
hparameters = {'stride':2,'f':3}
same1 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='SAME')
same2 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='VALID')

with tf.Session() as sess:
    a1 = sess.run(same1, feed_dict={tfx:x})
    a2 = sess.run(same2, feed_dict={tfx:x})
    print('origin\n',x[2,:,:,1])
    print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
    print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[1.1294839  0.37756377]
 [1.2245077  1.2245077 ]
 [0.16938244 0.40890053]] 
 shape (3, 3, 2, 2)
valid
 [[1.2245077]
 [1.2245077]] 
 shape (3, 2, 1, 2)


In [89]:
np.random.seed(1)
x = np.random.randn(3,5,4,2)
tfx = tf.placeholder(tf.float32,shape=(None,None,None,None))
hparameters = {'stride':2,'f':2}
same1 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='SAME')
same2 = tf.nn.max_pool(tfx,
                   ksize=(1,hparameters['f'],hparameters['f'],1),
                   strides=(1,hparameters['stride'],hparameters['stride'],1),
                   padding ='VALID')

with tf.Session() as sess:
    a1 = sess.run(same1, feed_dict={tfx:x})
    a2 = sess.run(same2, feed_dict={tfx:x})
    print('origin\n',x[2,:,:,1])
    print('same\n',a1[2,:,:,1],'\n','shape',a1.shape)
    print('valid\n',a2[2,:,:,1],'\n','shape',a2.shape)

origin
 [[-0.20075807  0.41005165  0.11900865  0.37756379]
 [ 1.12948391  0.18515642 -0.63873041  0.07734007]
 [ 0.04359686  0.69803203  1.2245077   0.59357852]
 [ 0.16938243 -0.9537006   0.03261455  0.31515939]
 [-0.85951594 -1.31228341 -1.61577235  0.40890054]]
same
 [[ 1.1294839   0.37756377]
 [ 0.698032    1.2245077 ]
 [-0.85951596  0.40890053]] 
 shape (3, 3, 2, 2)
valid
 [[1.1294839  0.37756377]
 [0.698032   1.2245077 ]] 
 shape (3, 2, 2, 2)



- [1,5,4,3]   
**valid**  n_h = ceil(h-f+1)/s  
**same**   n_h = ceil(h/s)  
**pad**    (h - f + pad)/s + 1 = n_h  -> pad = (n_h - 1)*s-h+f
- f=3, s=1
> valid 
[1, 3, 2, 3]   
> same 
[1, 5, 4, 3]
pad = ((1,1),(1,1))

- f=3, s=2
> valid 
[1, 2, 1, 3]  
> same 
[1, 3, 2, 3]
pad = ((1,1),(1,0))

- f=2, s=1  
> valid
[1, 4, 3, 3]  
> same
[1, 5, 4, 3]
pad = ((0,1),(0,1))  


- f=2, s=2  
> valid
[1, 2, 2, 3]  
> same
[1, 3, 2, 3]
pad = ((0,1),(0,1))


IndentationError: expected an indented block (<ipython-input-35-1260ede4d08b>, line 22)